In [1]:
import polars as pl

In [4]:
# joining
# default is inner
# then the usual suspects with left, right, full, cross, semi and anti

# we'll be using these two dfs
df_left = pl.DataFrame(
    {
        "key": ["A", "B", "C", "D"],
        "value": [1, 2, 3, 4]
    }
)

df_right = pl.DataFrame(
    {
        "key": ["B", "C", "D", "E"],
        "value": [5, 6, 7, 8],
    }
)

In [ ]:
# inner

df_left.join(
    df_right,
    on="key",
    how="inner"
)

key,value,value_right
str,i64,i64
"""B""",2,5
"""C""",3,6
"""D""",4,7


In [6]:
# what about with no how
df_left.join(
    df_right,
    on="key"
)

key,value,value_right
str,i64,i64
"""B""",2,5
"""C""",3,6
"""D""",4,7
